A text classification example for BERT.  
The data is from https://github.com/FudanNLP/nlpcc2017_news_headline_categorization.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
%tensorflow_version 2.x

import os
import warnings
import time
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn import preprocessing

os.chdir('./drive/My Drive/Python/Research/bert')
warnings.filterwarnings('ignore')

import mymodels as mm

In [2]:
MODEL = 'bert'
MODE = 'cls'
MAXLEN = 40
CATE = 18
DROP = 0.5
LRATE = 5e-5
BATCH = 64
EPOCH = 3
VOCAB = 'models/bert_base_ch/vocab.txt'
CONFIG = 'models/bert_base_ch/bert_config.json'
CKPT = 'models/bert_base_ch/bert_model.ckpt'

In [3]:
def data_processing(data, tokenizer, maxlen, batch, training):
  text1, seg1, mask1, label1 = [], [], [], []
  
  for i in range(len(data)):
    text2, seg2, mask2 = tokenizer.encoding(data['text'][i], None, maxlen)
    text1.append(text2)
    seg1.append(seg2)
    mask1.append(mask2)
    label1.append(data['label'][i])
    
  text1, seg1, mask1, label1 = np.array(text1), np.array(seg1), np.array(mask1), np.array(label1)
  data1 = tf.data.Dataset.from_tensor_slices((text1, seg1, mask1, label1))
  return data1.shuffle(len(text1)).batch(batch) if training else data1.batch(batch)


training_1 = pd.read_table('tasks/datasets/nlpcc_2017_news/train.txt', names=['label', 'text'])
dev_1 = pd.read_table('tasks/datasets/nlpcc_2017_news/dev.txt', names=['label', 'text'])
test_1 = pd.read_table('tasks/datasets/nlpcc_2017_news/test.txt', names=['label', 'text'])

label_1 = preprocessing.LabelEncoder()
training_1['label'] = label_1.fit_transform(training_1['label'])
dev_1['label'] = label_1.transform(dev_1['label'])
test_1['label'] = label_1.transform(test_1['label'])

tokenizer_1 = mm.Tokenizer()
tokenizer_1.loading(VOCAB)
training_2 = data_processing(training_1, tokenizer_1, MAXLEN, BATCH, True)
dev_2 = data_processing(dev_1, tokenizer_1, MAXLEN, BATCH, False)
test_2 = data_processing(test_1, tokenizer_1, MAXLEN, BATCH, False)

print(training_1.head())

   label                                               text
0      3      台 媒 预 测 周 冬 雨 金 马 奖 封 后 ， 大 气 的 倪 妮 却 佳 作 难 出
1      7  农 村 就 是 好 ， 能 吃 到 纯 天 然 无 添 加 的 野 生 蜂 蜜 ， 营 养 ...
2      5        1 4 款 知 性 美 装 ， 时 尚 惊 艳 搁 浅 的 阳 光 轻 熟 的 优 雅
3      9              火 焰 喷 射 器 1 0 0 0 度 火 焰 烧 死 鬼 子 4 连 拍
4     12                            1 8 岁 青 年 砍 死 8 8 岁 老 兵


In [4]:
class MyModel(keras.Model):
  def __init__(self, model, mode, config, drop, category):
    super(MyModel, self).__init__()
    self.bert = mm.BERT(config, model, mode)
    self.drop = keras.layers.Dropout(drop)
    self.dense = keras.layers.Dense(category, activation='softmax')

  def propagating(self, text, segment, mask, training):
    x1 = self.bert.propagating(text, segment, mask, training)
    return self.dense(self.drop(x1, training=training))


model_1 = MyModel(MODEL, MODE, CONFIG, DROP, CATE)
model_1.bert.loading(CKPT)
function_1 = keras.losses.SparseCategoricalCrossentropy()
optimizer_1 = mm.AdamWV2(EPOCH*(int(len(training_1)/BATCH)+1), LRATE)

loss_1 = tf.keras.metrics.Mean(name='training_loss')
acc_1 = tf.keras.metrics.SparseCategoricalAccuracy(name='training_accuracy')
acc_2 = tf.keras.metrics.SparseCategoricalAccuracy(name='dev_accuracy')

In [5]:
@tf.function
def step_training(text, segment, mask, y):
  with tf.GradientTape() as tape_1:
    pred_1 = model_1.propagating(text, segment, mask, True)
    value_1 = function_1(y, pred_1)

  grad_1 = tape_1.gradient(value_1, model_1.trainable_variables)
  grad_1, _ = tf.clip_by_global_norm(grad_1, 1.0)
  optimizer_1.apply_gradients(zip(grad_1, model_1.trainable_variables))
  loss_1(value_1)
  acc_1(y, pred_1)


@tf.function
def step_evaluating(text, segment, mask, y):
  pred_1 = model_1.propagating(text, segment, mask, False)
  acc_2(y, pred_1)


temp_1 = 'Training loss is {:.4f}, accuracy is {:.4f}.'
temp_2 = 'Dev accuracy is {:.4f}, and epoch cost is {:.4f}.'
count_1 = 0

for e_1 in range(EPOCH):
  print('Epoch {} running.'.format(e_1+1))
  time_0 = time.time()

  for x_1, x_2, x_3, y_1 in training_2:
    time_1, count_1 = time.time(), count_1+1
    step_training(x_1, x_2, x_3, y_1)

    if count_1 % 500 == 0:
      print(temp_1.format(float(loss_1.result()), float(acc_1.result())))

  for x_1, x_2, x_3, y_1 in dev_2:
    step_evaluating(x_1, x_2, x_3, y_1)
  
  print(temp_2.format(float(acc_2.result()), time.time()-time_0))
  print('**********')
  acc_1.reset_states()
  acc_2.reset_states()

Epoch 1 running.
Training loss is 1.5211, accuracy is 0.5865.
Training loss is 1.1668, accuracy is 0.6815.
Training loss is 1.0240, accuracy is 0.7184.
Training loss is 0.9330, accuracy is 0.7412.
Dev accuracy is 0.8236, and epoch cost is 931.2976.
**********
Epoch 2 running.
Training loss is 0.8732, accuracy is 0.8466.
Training loss is 0.8110, accuracy is 0.8523.
Training loss is 0.7622, accuracy is 0.8565.
Training loss is 0.7261, accuracy is 0.8576.
Training loss is 0.6960, accuracy is 0.8594.
Dev accuracy is 0.8395, and epoch cost is 920.6781.
**********
Epoch 3 running.
Training loss is 0.6677, accuracy is 0.9068.
Training loss is 0.6345, accuracy is 0.9085.
Training loss is 0.6063, accuracy is 0.9097.
Training loss is 0.5819, accuracy is 0.9115.
Training loss is 0.5605, accuracy is 0.9124.
Dev accuracy is 0.8410, and epoch cost is 918.4990.
**********


In [6]:
for x_1, x_2, x_3, y_1 in test_2:
  step_evaluating(x_1, x_2, x_3, y_1)

print('Test accuracy is {:.4f}.'.format(float(acc_2.result())))

Test accuracy is 0.8407.
